# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [67]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords



## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
#! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [2]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## your code to subset to one press release and take the string

pharma = doj.loc[doj['id'] == "17-1204"]['contents']

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [4]:
#A
pharma_cleaned = pharma.apply(lambda x: ''.join([char for char in x if char.isalpha() or char.isspace()]))

In [5]:
#B
pharma_str = pharma_cleaned.to_string()
tokens = word_tokenize(pharma_str)
tokens_pos = nltk.pos_tag(tokens)

In [6]:
#C

#create list of all adjectives
all_adj = [one_tok[0] for one_tok in tokens_pos if one_tok[1] == 'JJ']

#convert from list to dataframe
df_all_adj = pd.DataFrame(all_adj)
df_all_adj = df_all_adj.rename(columns={0:'Adjective'})

#sort by value counts
sorted_adj = df_all_adj['Adjective'].value_counts().reset_index()
sorted_adj.columns = ['Adjective','Count']

#find most 5 frequent 
most_freq_5 = sorted_adj.head(5) 
most_freq_5

,Adjective,Count
0,former,8
1,opioid,5
2,nationwide,4
3,addictive,3
4,other,3


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [7]:
#A #QUESTION: not sure if this we should be sotring the named entities somewhere? or if just printing them is ok? 
spacy_pharma = nlp(pharma.to_string())
for one_tok in spacy_pharma.ents: 
    print( "Entity: " + one_tok.text + " ; NER tag: " + one_tok.label_)

Entity: 4909 ; NER tag: DATE
Entity: Insys Therapeutics Inc. ; NER tag: ORG
Entity: today ; NER tag: DATE
Entity: Fentanyl ; NER tag: PERSON
Entity: More than 20,000 ; NER tag: CARDINAL
Entity: Americans ; NER tag: NORP
Entity: last year ; NER tag: DATE
Entity: millions ; NER tag: CARDINAL
Entity: Jeff Sessions ; NER tag: PERSON
Entity: This Justice Department ; NER tag: ORG
Entity: Trump ; NER tag: PERSON
Entity: American ; NER tag: NORP
Entity: ”John N. Kapoor ; NER tag: PERSON
Entity: 74 ; NER tag: DATE
Entity: Phoenix ; NER tag: GPE
Entity: Ariz. ; NER tag: GPE
Entity: the Board of Directors ; NER tag: ORG
Entity: Insys ; NER tag: ORG
Entity: this morning ; NER tag: TIME
Entity: Arizona ; NER tag: GPE
Entity: RICO ; NER tag: LAW
Entity: Kapoor ; NER tag: PERSON
Entity: Executive ; NER tag: ORG
Entity: Board ; NER tag: ORG
Entity: Insys ; NER tag: ORG
Entity: Phoenix ; NER tag: GPE
Entity: today ; NER tag: DATE
Entity: U.S. ; NER tag: GPE
Entity: District Court ; NER tag: ORG
Entity

In [8]:
#B
for one_tok in spacy_pharma.ents: 
    if (one_tok.label_ == "LAW"):
        print("Entity: " + one_tok.text + " ; NER tag: " + one_tok.label_)

Entity: RICO ; NER tag: LAW
Entity: the Controlled Substances Act ; NER tag: LAW
Entity: RICO ; NER tag: LAW


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

The Racketeer Influenced and Corrupt Organizations Act target organized crime, which can involve illicit drug activities such as moving money illegally via kickbacks from drug sales.  

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [9]:
#D
for one_tok in spacy_pharma.ents:
    if one_tok.label_ == "DATE":
        match = re.search(r'((?:.*?)(?:year|years)(?:.*?))', one_tok.text)
        if match:
            print("Entity: " + one_tok.text + " ; NER tag: " + one_tok.label_)


Entity: last year ; NER tag: DATE
Entity: 20 years ; NER tag: DATE
Entity: three years ; NER tag: DATE
Entity: five years ; NER tag: DATE
Entity: three years ; NER tag: DATE


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [10]:
press_pattern = r'(?<=\.|\s)(.*?((?:\d+ (?:year|years)).*?))(?=\.)'
press_pattern2 = r'((?:[^.]{0,50})(?:\d+ (?:year|years).*?\.)(?:.*?\.[^.]{0,50}))'
press_pattern3 = r'((?:.{0,50})(?:year|years)(?:.{0,50}))'


press_search = [re.findall(press_pattern3, spacy_pharma.text, re.IGNORECASE)]
press_search

[['Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet so',
  'each provide for a sentence of no greater than 20 years in prison, three years of supervised release and',
  'aw provide for a sentence of no greater than five years in prison, three years of supervised release and']]

Based on these other mentions of sentence and probation lengths, the CEO may be facing up to 20 years in prison with three years of probation if convicted after the indictment.

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [11]:
#A 

doj_subset = doj[(doj['topics_clean'] == 'Civil Rights') | 
                 (doj['topics_clean'] == 'Hate Crimes') | 
                 (doj['topics_clean'] == 'Project Safe Childhood')]

#Confirming it has 717 rows
doj_subset.shape


(717, 6)

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [14]:
def get_sentiment(press): 
    # Check if press is already a string
    if not isinstance(press, str):
        press = press.to_string()
    
    # create spaCy doc object from string 
    press_doc = nlp(press)
    
    # initialize sentiment analyzer
    sent_obj = SentimentIntensityAnalyzer()
    
    # create variable to store modified press
    mod_press = press
    
    # Create regex pattern for all named entities
    pattern = '|'.join([re.escape(entity.text) for entity in press_doc.ents])
    
    # Remove named entities from the press text
    mod_press = re.sub(pattern, '', mod_press)
    
    # Get sentiment score for modified_press 
    sentiment = sent_obj.polarity_scores(mod_press)
    
    return sentiment


In [13]:
#Apply to each press release in doj_subset 

for press_release in doj_subset['contents']: 
    get_sentiment(press_release)

{'neg': 0.2, 'neu': 0.751, 'pos': 0.049, 'compound': -0.9931}

{'neg': 0.134, 'neu': 0.797, 'pos': 0.069, 'compound': -0.9325}

{'neg': 0.092, 'neu': 0.832, 'pos': 0.076, 'compound': -0.7579}

{'neg': 0.127, 'neu': 0.788, 'pos': 0.085, 'compound': -0.9037}

{'neg': 0.179, 'neu': 0.777, 'pos': 0.044, 'compound': -0.9864}

...

{'neg': 0.155, 'neu': 0.781, 'pos': 0.064, 'compound': -0.9689}

{'neg': 0.084, 'neu': 0.816, 'pos': 0.099, 'compound': 0.5267}

{'neg': 0.159, 'neu': 0.751, 'pos': 0.089, 'compound': -0.9648}

{'neg': 0.138, 'neu': 0.762, 'pos': 0.1, 'compound': -0.9795}

{'neg': 0.152, 'neu': 0.817, 'pos': 0.031, 'compound': -0.9921}

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [23]:
doj_subset_wscore = doj_subset 

In [24]:
#C 

for index,row in doj_subset_wscore.iterrows():
    # Get the press release content from the 'contents' column
    press_release = row['contents']
    
    # Apply the sentiment analysis function to get sentiment scores
    sentiment_scores = get_sentiment(press_release)
    
    
    # Fill in the 'neg', 'neu', 'pos', and 'compound' columns with sentiment scores
    doj_subset_wscore.at[index, 'neg'] = sentiment_scores['pos']
    doj_subset_wscore.at[index, 'neu'] = sentiment_scores['neu']
    doj_subset_wscore.at[index, 'pos'] = sentiment_scores['pos']
    doj_subset_wscore.at[index, 'compound'] = sentiment_scores['compound']


In [25]:
doj_subset_wscore = doj_subset_wscore.sort_values(by='pos', ascending=True)
id_contents_neg = doj_subset_wscore.head(2)[['id', 'contents','neg']]
id_contents_neg

,id,contents,neg
5247,18-913,"Glenn Eugene Halfin, 64, from Grapevine, Texas, appeared today before U.S. Magistrate Judge Jeffrey L. Cureton in the U.S. District Court for the Northern District of Texas and pleaded guilty to a federal charge of interfering with an African-American family’s housing rights, announced Acting Assistant Attorney General John Gore of the Civil Rights Division and U.S. Attorney Erin Nealy Cox of the Northern District of Texas. According to court documents, Halfin threatened force, intimidated, and interfered with a family because of their race and occupancy of an apartment that was located directly above his own apartment. According to documents filed in connection with the guilty plea, on Dec. 19, 2017, Halfin purchased a baby doll at a Wal-Mart in Grapevine, Texas. He took a rope, fashioned it into a noose, and hung the baby doll from the noose. Halfin then hung the rope noose and baby doll on the railing directly in front of the only staircase the family could use to access their apartment. Halfin did so, knowing that this display would be particularly intimidating for the family who had a young daughter. In addition, the defendant referenced in his factual basis repeated intimidation of and interference with the same African-American family on other occasions. “The Justice Department will not tolerate acts of intimidation and fear, or illegal threats against any individual or family because of their race,” said Acting Assistant Attorney John Gore. “We will continue to prosecute hate crime offenders.” “No one should be afraid to go home at night,” said U.S. Attorney Erin Nealy Cox. “Our community will not tolerate crimes of intimidation or bigotry, and my office will continue to prosecute all those who persecute others based on their race, color, ethnicity, or religious beliefs.” Halfin faces a statutory maximum penalty of no more one year in federal prison and a $100,000 fine. His sentencing is scheduled for October 24. This case was investigated by the FBI and the Grapevine Police Department. The case was prosecuted by Trial Attorney Rebekah Bailey of the Civil Rights Division’s Criminal Section and Assistant United States Attorney Nicole Dana.",0.005
8075,11-1245,"WASHINGTON – Paul W. Miller, of Denham Springs, La., was convicted late yesterday of two counts of producing and one count of possessing child pornography, announced Assistant Attorney General Lanny A. Breuer of the Criminal Division and U.S. Attorney Donald J. Cazayoux Jr. of the Middle District of Louisiana. Miller, 44, was convicted by a federal jury following a two-day trial. U.S. District Judge James J. Brady presided over the trial. Evidence presented at trial showed that from October 2007 to May 2008, Miller sexually abused a 12-year-old girl and an 11-year-old girl and produced numerous photographs of the abuse. According to trial evidence, forensic examination of Miller’s computer revealed that Miller had used his computer to print and possess numerous images of child pornography, including both the images of child pornography he had produced and images of other child victims. Miller faces a maximum statutory sentence of 30 years in prison for each count of production of child pornography and 10 years in prison for the possession of child pornography count. The case is being prosecuted by Trial Attorney Alecia Riewerts Wolak of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Richard L. Bourgeois Jr. of the Middle District of Louisiana. The investigation was conducted by the FBI, the Denham Springs Police Department and the Louisiana Attorney General’s Office.",0.007


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [26]:
#D 
#Civil Rights, Hate Crimes, and Project Safe Childhood

#mean compound sentiment score for Civil Rights 
mean_civil = doj_subset_wscore.groupby(doj_subset_wscore['topics_clean'] == 'Civil Rights').agg({'compound': 'mean'})
print("the mean compound sentiment score for Civil Rights topics is:", mean_civil)

#mean compound sentiment score for Hate Crimes 
mean_hate = doj_subset_wscore.groupby(doj_subset_wscore['topics_clean'] == 'Hate Crimes')['compound'].mean()
print("the mean compound sentiment score for Hate Crimes topics is:", mean_hate)

#mean compound sentiment score for Project Safe Childhood
mean_project = doj_subset_wscore.groupby(doj_subset_wscore['topics_clean'] == 'Project Safe Childhood')['compound'].mean()
print("the mean compound sentiment score for Project Safe Childhood topics is:", mean_project)

the mean compound sentiment score for Civil Rights topics is:               compound
topics_clean          
False        -0.824946
True         -0.094465
the mean compound sentiment score for Hate Crimes topics is: topics_clean
False   -0.293930
True    -0.935971
Name: compound, dtype: float64
the mean compound sentiment score for Project Safe Childhood topics is: topics_clean
False   -0.470165
True    -0.660416
Name: compound, dtype: float64


**#E** One reason that we might see a variation in scores is that some topics may be talked about in a positive light in press releases (such as Project Safe Childhood), raising the mean compound score, whereas it would be unlikely to for a press release to talk about hate crimes in a positive way. 

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [68]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]
list_stopwords = stopwords.words("english")
list_stopwords_new = list_stopwords + custom_doj_stopwords
## initialize stemmer
snow_stemmer = SnowballStemmer(language='english')

In [91]:
#A 
def preprocess(string): 
    try:
        if not isinstance(string, str):
            string = string.to_string()
        string_lower = string.lower()
        nostop_string = [word 
                    for word in wordpunct_tokenize(string_lower) 
                    if word not in list_stopwords_new 
                        and word.isalpha()
                        and len(word) > 3]
        clean_string = [snow_stemmer.stem(word) for word in nostop_string]
        clean_string_str = " ".join(clean_string)
        return(clean_string_str)
    except:
        return("")
    

In [90]:
# testing the function on one contents instance
preprocess(doj_subset_wscore.iloc[1].contents)

'washington paul miller denham spring convict late yesterday count produc count possess child pornographi announc general lanni breuer crimin donald cazayoux middl louisiana miller convict feder juri follow judg jame bradi presid evid present show octob miller sexual abus year girl year girl produc numer photograph abus accord evid forens examin miller comput reveal miller use comput print possess numer imag child pornographi includ imag child pornographi produc imag child victim miller face maximum statutori sentenc year prison count product child pornographi year prison possess child pornographi count prosecut alecia riewert wolak crimin child exploit obscen section ceo richard bourgeoi middl louisiana conduct denham spring polic louisiana general'

In [95]:
#B
cleaned_strings = [preprocess(string) for string in 
                   doj_subset_wscore.contents]
doj_subset_wscore['processed_text'] = cleaned_strings
doj_subset_wscore

,id,title,contents,date,topics_clean,components_clean,neg,neu,pos,compound,processed_text
5247,18-913,Grapevine Texas Man Pleads Guilty to Federal Hate Crime Against an African-American Family,"Glenn Eugene Halfin, 64, from Grapevine, Texas, appeared today before U.S. Magistrate Judge Jeffrey L. Cureton in the U.S. District Court for the Northern District of Texas and pleaded guilty to a federal charge of interfering with an African-American family’s housing rights, announced Acting Assistant Attorney General John Gore of the Civil Rights Division and U.S. Attorney Erin Nealy Cox of the Northern District of Texas. According to court documents, Halfin threatened force, intimidated, and interfered with a family because of their race and occupancy of an apartment that was located directly above his own apartment. According to documents filed in connection with the guilty plea, on Dec. 19, 2017, Halfin purchased a baby doll at a Wal-Mart in Grapevine, Texas. He took a rope, fashioned it into a noose, and hung the baby doll from the noose. Halfin then hung the rope noose and baby doll on the railing directly in front of the only staircase the family could use to access their apartment. Halfin did so, knowing that this display would be particularly intimidating for the family who had a young daughter. In addition, the defendant referenced in his factual basis repeated intimidation of and interference with the same African-American family on other occasions. “The Justice Department will not tolerate acts of intimidation and fear, or illegal threats against any individual or family because of their race,” said Acting Assistant Attorney John Gore. “We will continue to prosecute hate crime offenders.” “No one should be afraid to go home at night,” said U.S. Attorney Erin Nealy Cox. “Our community will not tolerate crimes of intimidation or bigotry, and my office will continue to prosecute all those who persecute others based on their race, color, ethnicity, or religious beliefs.” Halfin faces a statutory maximum penalty of no more one year in federal prison and a $100,000 fine. His sentencing is scheduled for October 24. This case was investigated by the FBI and the Grapevine Police Department. The case was prosecuted by Trial Attorney Rebekah Bailey of the Civil Rights Division’s Criminal Section and Assistant United States Attorney Nicole Dana.",2018-07-12T00:00:00-04:00,Hate Crimes,Civil Rights Division; Civil Rights - Criminal Section,0.005,0.806,0.005,-0.9955,glenn eugen halfin grapevin texa appear today magistr judg jeffrey cureton court northern texa plead guilti feder charg interf african american famili hous announc act general john gore erin neali northern texa accord court document halfin threaten forc intimid interf famili race occup apart locat direct apart accord document file connect guilti plea halfin purchas babi doll mart grapevin texa took rope fashion noos hung babi doll noos halfin hung rope noos babi doll rail direct front staircas famili could access apart halfin know display would particular intimid famili young daughter addit defend referenc factual basi repeat intimid interfer african american famili occas toler act intimid fear illeg threat individu famili race said act john gore continu prosecut hate crime offend afraid home night said erin neali communiti toler crime intimid bigotri continu prosecut persecut other base race color ethnic religi belief halfin face statutori maximum penalti year feder prison fine sentenc schedul octob investig grapevin polic prosecut rebekah bailey crimin section unit state nicol dana
8075,11-1245,Louisiana Man Convicted of Producing and Possessing Child Pornography,"WASHINGTON – Paul W. Miller, of Denham Springs, La., was convicted late yesterday of two counts of producing and one count of possessing child pornography, announced Assistant Attorney General Lanny A. Breuer of the Criminal Division and U.S. Attorney Donald J. Cazayoux Jr. of the Middle District of Louisiana. Mill

In [101]:
## your code showing the examples

doj_subset_wscore[(doj_subset_wscore['id'] == '16-718') | (doj_subset_wscore['id'] == '16-217')][['id', 'contents', 'processed_text']]


,id,contents,processed_text
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",nine count indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison free cruel unusu punish sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont pate charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three lie convict marsher sturdiv face maximum sentenc year prison excess forc charg three offic face five year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti investig jackson cooper mississippi correct prosecut robert coleman northern mississippi dana mulhaus crimin section marsher indict
6727,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [167]:

def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out()) #had to edit from get_feature_names because version difference
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [178]:
# your code here

doj_subset_wscore.rename(columns={'compound': 'Compound'}, inplace=True)

doj_subset_wscore.rename_axis('Index', inplace=True)

dtm_doj = create_dtm(list_of_strings= doj_subset_wscore.processed_text,
                metadata = 
                doj_subset_wscore[['id','Compound', 'topics_clean']])



In [179]:
list1 = dtm_doj.keys().to_list()

In [180]:
seen = set()
dupes = [x for x in list1 if x in seen or seen.add(x)]    
dupes

[]

In [204]:
def get_topwords(data, subset, decimal=None, specific_value=None):
    if decimal is not None:
        press = data[data[subset] > data[subset].quantile(decimal)]
    else:
        press = data
        
    if specific_value is not None:
        press = press[press[subset] == specific_value]
        
    top_terms = press[press.columns[4:]].sum(axis=0)
    return top_terms.sort_values(ascending=False).head(10)

In [205]:
#B

top_positive_words = get_topwords(data=dtm_doj, subset='Compound', decimal=0.95)
top_positive_words

agreement     175
enforc        121
state         118
ensur         110
disabl        107
communiti      97
court          91
servic         90
student        87
settlement     87
dtype: int64

In [207]:
#C

top_negative_words = get_topwords(data=dtm_doj, subset='Compound', decimal=0.05)
top_negative_words

child       1079
feder       1070
victim       996
prosecut     948
sentenc      904
said         896
state        881
charg        857
year         834
general      817
dtype: int64

In [203]:
dtm_doj.topics_clean.unique()

array(['Hate Crimes', 'Project Safe Childhood', 'Civil Rights'],
      dtype=object)

In [214]:
#D

top_hatecrime_words = get_topwords(data=dtm_doj, subset='topics_clean', specific_value='Hate Crimes')
top_hatecrime_words

top_childhood_words = get_topwords(data=dtm_doj, subset='topics_clean', specific_value='Project Safe Childhood')
top_childhood_words

top_civilrights_words = get_topwords(data=dtm_doj, subset='topics_clean', specific_value='Civil Rights')
top_civilrights_words

victim      591
crime       557
hate        524
defend      484
prosecut    478
charg       463
sentenc     455
american    451
feder       432
guilti      430
dtype: int64

child          1022
exploit         701
sexual          572
safe            479
childhood       474
project         472
pornographi     452
children        423
crimin          405
prosecut        374
dtype: int64

offic        637
hous         633
discrimin    616
enforc       544
disabl       532
said         497
feder        479
violat       477
state        452
court        414
dtype: int64

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [224]:
# A
## Step 1: re-tokenize and store in list using the preprocessed texts

text_raw_tokens = [wordpunct_tokenize(one_text) for one_text in 
                  doj_subset_wscore['processed_text']]


## Step 2: use gensim create dictionary - gets all unique words across documents
text_raw_dict = corpora.Dictionary(text_raw_tokens)
raw_len = len(text_raw_dict) # get length for comparison below

### explore first few keys and values
### see that key is just an arbitrary counter; value is the word itself
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]}


## Step 3: filter out very rare and very common words
## here, we are using the threshold that a word needs to appear in at least
## 5% of docs but not more than 95%
## this is an integer count of docs so we round
lower_bound = round(doj_subset_wscore.shape[0]*0.05)
upper_bound = round(doj_subset_wscore.shape[0]*0.95)

### apply filtering to dictionary
text_raw_dict.filter_extremes(no_below = lower_bound,
                             no_above = upper_bound)
print(f'Filtering out very rare and very common words reduced the \
length of dictionary from {str(raw_len)} to {str(len(text_raw_dict))}.')
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]} # show first five entries after filtering


## Step 4: apply dictionary to TOKENIZED texts
## this creates a mapping between each word 
## in a specific listing and the key in the dictionary.
## for words that remain in the filtered dictionary,
## output is a list where len(list) == n documents
## and each element in the list is a list of tuples
## containing the mappings
corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]

{0: 'access', 1: 'accord', 2: 'act', 3: 'addit', 4: 'afraid'}

Filtering out very rare and very common words reduced the length of dictionary from 6866 to 623.


{0: 'access', 1: 'accord', 2: 'act', 3: 'addit', 4: 'african'}

In [245]:
## Step 5: estimate the model
## full documentation here - https://radimrehurek.com/gensim/models/ldamodel.html
## here, we're feed the lda function (1) the corpus we created from the dictionary
## (2) a parameter we decide on for the number of topics,
## (3) the dictionary itself,
## (4) parameter for number of passes through training data
## (5) parameter that returns, for each word remaining in dict, the 
## topic probabilities
## see documentation for many other arguments you can vary
num_topics = 3
ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                         num_topics = num_topics, 
                                         id2word=text_raw_dict, 
                                         passes=6, 
                                         alpha = 'auto',
                                         per_word_topics = True, 
                                         random_state = 91988)

In [237]:
### print topics and words
topics = ldamod.print_topics(num_words = 15)
counter = 1
for topic in topics:
    print("Topic " + str(counter))
    print(topic)
    counter += 1

Topic 1
(0, '0.018*"discrimin" + 0.017*"hous" + 0.015*"disabl" + 0.011*"enforc" + 0.011*"agreement" + 0.011*"state" + 0.010*"court" + 0.010*"said" + 0.009*"alleg" + 0.009*"requir" + 0.009*"settlement" + 0.009*"feder" + 0.008*"fair" + 0.008*"inform" + 0.008*"violat"')
Topic 2
(1, '0.015*"victim" + 0.014*"charg" + 0.013*"prosecut" + 0.013*"sentenc" + 0.013*"defend" + 0.013*"feder" + 0.013*"crime" + 0.012*"said" + 0.012*"guilti" + 0.011*"hate" + 0.010*"year" + 0.010*"indict" + 0.010*"american" + 0.010*"investig" + 0.010*"prison"')
Topic 3
(2, '0.037*"child" + 0.025*"exploit" + 0.022*"sexual" + 0.017*"safe" + 0.017*"project" + 0.017*"childhood" + 0.016*"pornographi" + 0.015*"children" + 0.015*"crimin" + 0.014*"prosecut" + 0.013*"sentenc" + 0.013*"victim" + 0.011*"ceo" + 0.011*"year" + 0.011*"minor"')


In [222]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
pyLDAvis.enable_notebook()

In [223]:
lda_display = gensimvis.prepare(ldamod, corpus_fromdict, text_raw_dict)
pyLDAvis.display(lda_display)

## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [241]:
topic_probs_bydoc = [ldamod.get_document_topics(item, minimum_probability=0) for item in corpus_fromdict]

In [243]:

# Write an assert statement to check the length of topic_probs_bydoc
assert len(topic_probs_bydoc) == len(doj_subset_wscore), "Length of topic probabilities and doj_subset_wscore DataFrame rows are not equal"

# Print a message if the assertion passes
print("Assertion passed: Length of topic probabilities equals the number of rows in doj_subset_wscore DataFrame.")


Assertion passed: Length of topic probabilities equals the number of rows in doj_subset_wscore DataFrame.


In [252]:
for topic_id in range(num_topics):
    col_name = "topic_" + str(topic_id + 1) + "_prob"
    doj_subset_wscore[col_name] = [doc_probs[topic_id][1] if len(doc_probs) > topic_id else 0 for doc_probs in topic_probs_bydoc]

In [253]:
# List to store the top topic for each document
top_topics = []

# Iterate through the topic probabilities for each document
for doc_probs in topic_probs_bydoc:
    # Find the topic with the highest probability
    top_topic = max(doc_probs, key=lambda x: x[1])[0]
    top_topics.append(top_topic)

# Top_topic column based on the highest-probability topic for each document
doj_subset_wscore['top_topic'] = top_topics

doj_subset_wscore

,id,title,contents,date,topics_clean,components_clean,neg,neu,pos,Compound,processed_text,topic_1_prob,topic_2_prob,topic_3_prob,top_topic
Index,,,,,,,,,,,,,,,
5247,18-913,Grapevine Texas Man Pleads Guilty to Federal Hate Crime Against an African-American Family,"Glenn Eugene Halfin, 64, from Grapevine, Texas, appeared today before U.S. Magistrate Judge Jeffrey L. Cureton in the U.S. District Court for the Northern District of Texas and pleaded guilty to a federal charge of interfering with an African-American family’s housing rights, announced Acting Assistant Attorney General John Gore of the Civil Rights Division and U.S. Attorney Erin Nealy Cox of the Northern District of Texas. According to court documents, Halfin threatened force, intimidated, and interfered with a family because of their race and occupancy of an apartment that was located directly above his own apartment. According to documents filed in connection with the guilty plea, on Dec. 19, 2017, Halfin purchased a baby doll at a Wal-Mart in Grapevine, Texas. He took a rope, fashioned it into a noose, and hung the baby doll from the noose. Halfin then hung the rope noose and baby doll on the railing directly in front of the only staircase the family could use to access their apartment. Halfin did so, knowing that this display would be particularly intimidating for the family who had a young daughter. In addition, the defendant referenced in his factual basis repeated intimidation of and interference with the same African-American family on other occasions. “The Justice Department will not tolerate acts of intimidation and fear, or illegal threats against any individual or family because of their race,” said Acting Assistant Attorney John Gore. “We will continue to prosecute hate crime offenders.” “No one should be afraid to go home at night,” said U.S. Attorney Erin Nealy Cox. “Our community will not tolerate crimes of intimidation or bigotry, and my office will continue to prosecute all those who persecute others based on their race, color, ethnicity, or religious beliefs.” Halfin faces a statutory maximum penalty of no more one year in federal prison and a $100,000 fine. His sentencing is scheduled for October 24. This case was investigated by the FBI and the Grapevine Police Department. The case was prosecuted by Trial Attorney Rebekah Bailey of the Civil Rights Division’s Criminal Section and Assistant United States Attorney Nicole Dana.",2018-07-12T00:00:00-04:00,Hate Crimes,Civil Rights Division; Civil Rights - Criminal Section,0.005,0.806,0.005,-0.9955,glenn eugen halfin grapevin texa appear today magistr judg jeffrey cureton court northern texa plead guilti feder charg interf african american famili hous announc act general john gore erin neali northern texa accord court document halfin threaten forc intimid interf famili race occup apart locat direct apart accord document file connect guilti plea halfin purchas babi doll mart grapevin texa took rope fashion noos hung babi doll noos halfin hung rope noos babi doll rail direct front staircas famili could access apart halfin know display would particular intimid famili young daughter addit defend referenc factual basi repeat intimid interfer african american famili occas toler act intimid fear illeg threat individu famili race said act john gore continu prosecut hate crime offend afraid home night said erin neali communiti toler crime intimid bigotri continu prosecut persecut other base race color ethnic religi belief halfin face statutori maximum penalti year feder prison fine sentenc schedul octob investig grapevin polic prosecut rebekah bailey crimin section unit state nicol dana,0.000479,0.999077,0.000444,1
8075,11-1245,Louisiana Man Convicted of Producing and Possessing Child Pornography,"WASHINGTON – Paul W. Miller, of Denham Springs, La., was convicted late yesterday of two counts of producing and one count of possessing child pornography, announced Assistant Attorney General Lanny A. Breuer of the C

In [ ]:
## your code here to add those topic probabilities to the dataframe

In [ ]:
## your code here to summarize the topic proportions for each of the topics_clean 

In [254]:
#C
pd.crosstab(doj_subset_wscore['topics_clean'], doj_subset_wscore['top_topic'], normalize='index')

top_topic,0,1,2
topics_clean,,,
Civil Rights,0.659016,0.340984,0.000000
Hate Crimes,0.000000,1.000000,0.000000
Project Safe Childhood,0.006024,0.000000,0.993976


**write interpretation**

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [260]:
doj_subset_wscore.columns

Index(['id', 'title', 'contents', 'date', 'topics_clean', 'components_clean',
       'neg', 'neu', 'pos', 'Compound', 'processed_text', 'topic_1_prob',
       'topic_2_prob', 'topic_3_prob', 'top_topic'],
      dtype='object')

In [263]:
bigrams = []

def create_bigram_onedoc(processed_text):
    tokens = processed_text.split()
    
    for first, second in zip(tokens, tokens[1:]):
        string = first + "_" + second
        bigrams.append(string)
    
    return bigrams

text_to_process = doj_subset_wscore.iloc[1]['processed_text']
create_bigram_onedoc(text_to_process)


['washington_paul',
 'paul_miller',
 'miller_denham',
 'denham_spring',
 'spring_convict',
 'convict_late',
 'late_yesterday',
 'yesterday_count',
 'count_produc',
 'produc_count',
 'count_possess',
 'possess_child',
 'child_pornographi',
 'pornographi_announc',
 'announc_general',
 'general_lanni',
 'lanni_breuer',
 'breuer_crimin',
 'crimin_donald',
 'donald_cazayoux',
 'cazayoux_middl',
 'middl_louisiana',
 'louisiana_miller',
 'miller_convict',
 'convict_feder',
 'feder_juri',
 'juri_follow',
 'follow_judg',
 'judg_jame',
 'jame_bradi',
 'bradi_presid',
 'presid_evid',
 'evid_present',
 'present_show',
 'show_octob',
 'octob_miller',
 'miller_sexual',
 'sexual_abus',
 'abus_year',
 'year_girl',
 'girl_year',
 'year_girl',
 'girl_produc',
 'produc_numer',
 'numer_photograph',
 'photograph_abus',
 'abus_accord',
 'accord_evid',
 'evid_forens',
 'forens_examin',
 'examin_miller',
 'miller_comput',
 'comput_reveal',
 'reveal_miller',
 'miller_use',
 'use_comput',
 'comput_print',
 'pri

In [266]:
doj_subset_wscore['processed_text_bigrams'] = doj_subset_wscore['processed_text'].apply(create_bigram_onedoc)
doj_subset_wscore

#for some reason, it's applying the function and matching to the wrong row

,id,title,contents,date,topics_clean,components_clean,neg,neu,pos,Compound,processed_text,topic_1_prob,topic_2_prob,topic_3_prob,top_topic,processed_text_bigrams
Index,,,,,,,,,,,,,,,,
5247,18-913,Grapevine Texas Man Pleads Guilty to Federal Hate Crime Against an African-American Family,"Glenn Eugene Halfin, 64, from Grapevine, Texas, appeared today before U.S. Magistrate Judge Jeffrey L. Cureton in the U.S. District Court for the Northern District of Texas and pleaded guilty to a federal charge of interfering with an African-American family’s housing rights, announced Acting Assistant Attorney General John Gore of the Civil Rights Division and U.S. Attorney Erin Nealy Cox of the Northern District of Texas. According to court documents, Halfin threatened force, intimidated, and interfered with a family because of their race and occupancy of an apartment that was located directly above his own apartment. According to documents filed in connection with the guilty plea, on Dec. 19, 2017, Halfin purchased a baby doll at a Wal-Mart in Grapevine, Texas. He took a rope, fashioned it into a noose, and hung the baby doll from the noose. Halfin then hung the rope noose and baby doll on the railing directly in front of the only staircase the family could use to access their apartment. Halfin did so, knowing that this display would be particularly intimidating for the family who had a young daughter. In addition, the defendant referenced in his factual basis repeated intimidation of and interference with the same African-American family on other occasions. “The Justice Department will not tolerate acts of intimidation and fear, or illegal threats against any individual or family because of their race,” said Acting Assistant Attorney John Gore. “We will continue to prosecute hate crime offenders.” “No one should be afraid to go home at night,” said U.S. Attorney Erin Nealy Cox. “Our community will not tolerate crimes of intimidation or bigotry, and my office will continue to prosecute all those who persecute others based on their race, color, ethnicity, or religious beliefs.” Halfin faces a statutory maximum penalty of no more one year in federal prison and a $100,000 fine. His sentencing is scheduled for October 24. This case was investigated by the FBI and the Grapevine Police Department. The case was prosecuted by Trial Attorney Rebekah Bailey of the Civil Rights Division’s Criminal Section and Assistant United States Attorney Nicole Dana.",2018-07-12T00:00:00-04:00,Hate Crimes,Civil Rights Division; Civil Rights - Criminal Section,0.005,0.806,0.005,-0.9955,glenn eugen halfin grapevin texa appear today magistr judg jeffrey cureton court northern texa plead guilti feder charg interf african american famili hous announc act general john gore erin neali northern texa accord court document halfin threaten forc intimid interf famili race occup apart locat direct apart accord document file connect guilti plea halfin purchas babi doll mart grapevin texa took rope fashion noos hung babi doll noos halfin hung rope noos babi doll rail direct front staircas famili could access apart halfin know display would particular intimid famili young daughter addit defend referenc factual basi repeat intimid interfer african american famili occas toler act intimid fear illeg threat individu famili race said act john gore continu prosecut hate crime offend afraid home night said erin neali communiti toler crime intimid bigotri continu prosecut persecut other base race color ethnic religi belief halfin face statutori maximum penalti year feder prison fine sentenc schedul octob investig grapevin polic prosecut rebekah bailey crimin section unit state nicol dana,0.000479,0.999077,0.000444,1,"[washington_paul, paul_miller, miller_denham, denham_spring, spring_convict, convict_late, late_yesterday, yesterday_count, count_produc, produc_count, count_possess, possess_child, child_pornographi, pornographi_announc, announc_general, general_lanni, lanni_breuer, breuer_c

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [ ]:
# your code here

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [ ]:
# your code here 